## Kural Tabanlı Sınıflandırma ile Potansiyel Müşteri Getirisi Hesaplama

Bir oyun şirketi müşterilerinin bazı özelliklerini kullanarak
    seviye tabanlı (level based) yeni müşteri tanımları
    (persona) oluşturmak ve bu yeni müşteri tanımlarına
    göre segmentler oluşturup bu segmentlere göre yeni
    gelebilecek müşterilerin şirkete ortalama ne kadar
    kazandırabileceğini tahmin etmek istemektedir.

Örnek:
    Türkiye’den IOS kullanıcısı olan 25 yaşındaki bir erkek
    kullanıcının ortalama ne kadar kazandırabileceği
    belirlenmek isteniyor.

Veri Seti Hikayesi :

    Persona.csv veri seti uluslararası bir oyun şirketinin sattığı ürünlerin fiyatlarını
    ve bu ürünleri satın alan kullanıcıların bazı demografik bilgilerini
    barındırmaktadır.
    Veri seti her satış işleminde oluşan kayıtlardan meydana gelmektedir.

    Bunun anlamı tablo tekilleştirilmemiştir.
    Diğer bir ifade ile belirli demografik özelliklere sahip bir kullanıcı birden fazla
    alışveriş yapmış olabilir.

    PRICE – Müşterinin harcama tutarı
    SOURCE – Müşterinin bağlandığı cihaz türü
    SEX – Müşterinin cinsiyeti
    COUNTRY – Müşterinin ülkesi
    AGE – Müşterinin yaşı

In [2]:
import pandas as pd

def check_df(dataframe, head=5):
    print("##################### Shape #####################")
    print(dataframe.shape)
    print("##################### Types #####################")
    print(dataframe.dtypes)
    print("##################### Head #####################")
    print(dataframe.head(head))
    print("##################### NA #####################")
    print(dataframe.isnull().sum())
    print("##################### Quantiles #####################")
    print(dataframe.describe().T)

df = pd.read_csv("persona.csv")
check_df(df)

##################### Shape #####################
(5000, 5)
##################### Types #####################
PRICE       int64
SOURCE     object
SEX        object
COUNTRY    object
AGE         int64
dtype: object
##################### Head #####################
   PRICE   SOURCE   SEX COUNTRY  AGE
0     39  android  male     bra   17
1     39  android  male     bra   17
2     49  android  male     bra   17
3     29  android  male     tur   17
4     49  android  male     tur   17
##################### NA #####################
PRICE      0
SOURCE     0
SEX        0
COUNTRY    0
AGE        0
dtype: int64
##################### Quantiles #####################
        count     mean        std   min   25%   50%   75%   max
PRICE  5000.0  34.1320  12.464897   9.0  29.0  39.0  39.0  59.0
AGE    5000.0  23.5814   8.995908  15.0  17.0  21.0  27.0  66.0


   * 5000 satır ve 5 değişkenden oluşuyor.

   * PRICE,AGE değişkeni tipi int64
    SOURCE,SEX,COUNTRY kategorik değişkenler

   * Hiçbir değişkende bir eksik veri bulunmuyor.

In [5]:
df["SOURCE"].nunique()
df["SOURCE"].value_counts()

android    2974
ios        2026
Name: SOURCE, dtype: int64

 * 2 farklı unque değeri vardır.
    

In [6]:
df["PRICE"].value_counts()

29    1305
39    1260
49    1031
19     992
59     212
9      200
Name: PRICE, dtype: int64

 * 6 farklı unque değeri vardır

### Ülkelere göre satışlardan toplam ne kadar kazanılmış ve kaç satış yapılıştır ?

In [8]:
df[["COUNTRY","PRICE"]].groupby("COUNTRY").agg({"count","sum"})

PRICE      
           sum count
COUNTRY             
bra      51354  1496
can       7730   230
deu      15485   455
fra      10177   303
tur      15689   451
usa      70225  2065

###  source türlerinde göre satış sayıları nedir ?

In [9]:
df[["SOURCE","PRICE"]].groupby("SOURCE").agg({"count"})

,PRICE
,count
SOURCE,
android,2974
ios,2026


### Ülkelere göre PRICE ortalamaları nedir ?

In [10]:
df[["COUNTRY","PRICE"]].groupby("COUNTRY").agg({"mean"})


,PRICE
,mean
COUNTRY,
bra,34.327540
can,33.608696
deu,34.032967
fra,33.587459
tur,34.787140
usa,34.007264


### SOURCE'lara göre PRICE ortalamaları nedir ?

In [11]:
df[["SOURCE","PRICE"]].groupby("SOURCE").agg({"mean"})


,PRICE
,mean
SOURCE,
android,34.174849
ios,34.069102


### COUNTRY - SOURCE kırılımında PRICE ortalamaları nedir ?

In [12]:
df[["SOURCE","COUNTRY","PRICE"]].groupby(["COUNTRY","SOURCE"]).agg({"mean"})


PRICE
                      mean
COUNTRY SOURCE            
bra     android  34.387029
        ios      34.222222
can     android  33.330709
        ios      33.951456
deu     android  33.869888
        ios      34.268817
fra     android  34.312500
        ios      32.776224
tur     android  36.229437
        ios      33.272727
usa     android  33.760357
        ios      34.371703

In [15]:
df.groupby(["COUNTRY","SOURCE","SEX","AGE"]).agg({"mean"})

PRICE
                                 mean
COUNTRY SOURCE  SEX    AGE           
bra     android female 15   38.714286
                       16   35.944444
                       17   35.666667
                       18   32.255814
                       19   35.206897
...                               ...
usa     ios     male   42   30.250000
                       50   39.000000
                       53   34.000000
                       55   29.000000
                       59   46.500000

[348 rows x 1 columns]

### Bu kısımda verileri kural tabanlı sınıflandıracağız ve sonrasında potasiyel müşteri getirisi hesaplıyacağız

In [16]:
agg_df = pd.DataFrame(df.groupby(["COUNTRY","SOURCE","SEX","AGE"]).agg({"mean"})).reset_index()
agg_df.columns = ["COUNTRY","SOURCE","SEX","AGE","PRICE"]
agg_df.sort_values("PRICE", ascending=False,inplace= True)

In [17]:
agg_df["AGE"].max()

66

In [18]:
agg_df["AGE"].min()

15

In [19]:
#0_18 , 19_23 , 24_30 , 31_40 , 41+
agg_df["AGE_CAT"] = pd.cut(agg_df["AGE"],[0,18,23,30,40,66], labels=["1_18","19_23","24_30","31_40","41+"], ordered=False)

In [23]:
agg_df

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT
47,bra,android,male,46,59.0,41+
290,usa,android,male,36,59.0,31_40
167,fra,android,female,24,59.0,24_30
339,usa,ios,male,32,54.0,31_40
125,deu,android,female,36,49.0,31_40
...,...,...,...,...,...,...
319,usa,ios,female,38,19.0,31_40
315,usa,ios,female,30,19.0,24_30
97,can,android,female,27,19.0,24_30
171,fra,android,male,18,19.0,1_18


In [ ]:
agg_df = pd.DataFrame([{"customers_level_based":str(i[0]+"_"+i[1]+"_"+i[2]+"_"+i[5]).upper(),"PRICE": i[4]} for i in agg_df.values])

In [29]:
agg_df

,customers_level_based,PRICE
0,BRA_ANDROID_MALE_41+,59.0
1,USA_ANDROID_MALE_31_40,59.0
2,FRA_ANDROID_FEMALE_24_30,59.0
3,USA_IOS_MALE_31_40,54.0
4,DEU_ANDROID_FEMALE_31_40,49.0
...,...,...
343,USA_IOS_FEMALE_31_40,19.0
344,USA_IOS_FEMALE_24_30,19.0
345,CAN_ANDROID_FEMALE_24_30,19.0
346,FRA_ANDROID_MALE_1_18,19.0


In [31]:
agg_df = agg_df[:].groupby("customers_level_based").agg({"PRICE":"mean"})

In [32]:
agg_df["SEGMENT"] = pd.cut(agg_df["PRICE"],4, labels=["D","C","B","A"])
agg_df = agg_df.reset_index()

In [33]:
agg_df[:].groupby("SEGMENT").agg({"PRICE":["mean","max","sum"]}).reset_index()

SEGMENT      PRICE                        
                mean        max          sum
0       D  22.502193  25.052632   112.510965
1       C  30.588433  32.014706   672.945535
2       B  34.732598  37.685764  2396.549258
3       A  40.784540  45.428571   530.199023

In [34]:
agg_df[agg_df["SEGMENT"] == "C"].agg({"PRICE":["mean","max","min","std","sum"]})

,PRICE
mean,30.588433
max,32.014706
min,27.333333
std,1.377497
sum,672.945535


In [35]:
def yeni_musteri(dataframe,new_user):
    print(dataframe[dataframe["customers_level_based"] == new_user])

### 33 yaşında ANDROID kullanan bir Türk kadını hangi segmente aittir ve ortalama ne kadar gelir kazanması beklenir?

In [37]:
yeni_musteri(agg_df,"TUR_ANDROID_FEMALE_31_40")

       customers_level_based      PRICE SEGMENT
72  TUR_ANDROID_FEMALE_31_40  41.833333       A


A segmentine ait ve ortalama 41,8 gelir kazandırması bekleniyor.

### 35 yaşında IOS kullanan bir fıransız kadını hangi segmente ve ortalama ne kadar gelir kazanması beklenir ?

In [38]:
yeni_musteri(agg_df,"FRA_IOS_FEMALE_31_40")

   customers_level_based      PRICE SEGMENT
63  FRA_IOS_FEMALE_31_40  32.818182       B


B segmentine aittir ve ortalama 32,8 gelir kazndırması bekleniyor.